# Imports

In [1]:
!pip install gradio duckduckgo-images-api huggingface_hub transformers gradio_pdf PyMuPDF torch -U bitsandbytes PyPDF2 pytesseract loguru bing_image_downloader

In [2]:
from transformers import BitsAndBytesConfig # used when quatizing models
import torch
from threading import Thread
from transformers import TextIteratorStreamer

In [3]:
from huggingface_hub import login # used to login into huggingFace
from transformers import AutoTokenizer, AutoModelForCausalLM # to access llm models
import gradio as gr # ui interface
from PIL import Image # to open images
import os # used to manipulate files/directories
import time
import uuid # used in giving unique names to each file to generate a unique file path
import pymupdf # to open pdf document
from gradio_pdf import PDF # to show pdf in the gradio interface
from PyPDF2 import PdfReader # used to read the PDF
from pytesseract import image_to_string # used to extract text from images

In [4]:
#from pathlib import Path
from loguru import logger # for logging errors
from bing_image_downloader import downloader # to download images from the web
import shutil # to delete the directory created immediately after serving its purpose

# Signing into Hugging Face

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read)

# Defining models, Quantizing and creating Tokenizer for selected model

In [6]:
mistral="mistralai/Mistral-7B-Instruct-v0.1" # too heavy for runtime (Crashed CPU runtime)
allenai="allenai/OLMoE-1B-7B-0125-Instruct"
llama="meta-llama/Llama-3.3-70B-Instruct"
qwenai72B="Qwen/Qwen2.5-72B-Instruct"
qwenai7B = "Qwen/Qwen2-7B-Instruct"
deepseek="deepseek-ai/DeepSeek-R1"
GEMMA2 = "google/gemma-2-2b-it" # smallest model, fit well with compute size

In [7]:
# performing quantization on model
quant_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  #bnb_4bit_compute_dtype=torch.float32,  # Use a CPU-compatible dtype
  bnb_4bit_compute_dtype=torch.bfloat16, # use if CUDA is available on system
  bnb_4bit_quant_type="nf4"
)

# declaring tokenizer for chosen llm
tokenizer = AutoTokenizer.from_pretrained(GEMMA2)

# Display and Extracting Text From Files (PDF, .docx & images)

In [8]:
def display_pdf(file_path):
    """
    This function simply returns the file path for PDF display.
    If it's a Word document, it converts it to PDF first.
    """
    if file_path is None:
        return None

    # Check if file is a PDF or needs conversion
    file_ext = os.path.splitext(file_path)[1].lower()

    if file_ext == '.pdf':
        # If already a PDF, just return the path
        return file_path
    elif file_ext == '.docx':
        # If it's a Word document, convert to PDF
        pdf_path = os.path.splitext(file_path)[0] + "_converted.pdf"

        try:
            with pymupdf.open(file_path) as doc:
                pdf_bytes = doc.convert_to_pdf()

                with open(pdf_path, 'wb') as pdf_file:
                    pdf_file.write(pdf_bytes)

            return pdf_path
        except Exception as e:
            print(f"Error converting document to PDF: {e}")
            return None
    else:
        # Unsupported file type
        print(f"Unsupported file type: {file_ext}")
        return None

In [9]:
def extract_text_from_pdf(file_path):
    pdf_reader = PdfReader(file_path)

    raw_text = ''

    for i, page in enumerate(pdf_reader.pages):

        text = page.extract_text()
        if text:
            raw_text += text

    return raw_text

In [10]:
def extract_text(file_path):
    with pymupdf.open(file_path) as f:
      if f.is_pdf:
          return extract_text_from_pdf(file_path)
      else:
        pdf_bytes = f.convert_to_pdf()
        unique_filename = f"{uuid.uuid4()}.pdf" # gives file a unique name

        tmp_file_path = os.path.join(os.path.dirname(file_path), unique_filename) # creates a location for new file

        try:
            with open(tmp_file_path, 'wb') as tmp_file:
                tmp_file.write(pdf_bytes)

            text = extract_text_from_pdf(tmp_file_path)

            return text
        finally:
            # Always clean up temporary file
            if os.path.exists(tmp_file_path):
                os.remove(tmp_file_path)

In [11]:
#text_with_pyPDF = extract_text("/content/Screenshot 2025-01-17 at 17.45.18.png")
#print(text_with_pyPDF)

In [12]:
def extract_text_from_img(online_resume):
    image_content = []
    raw_text = str(image_to_string(online_resume))
    image_content.append(raw_text)

    return "\n".join(image_content)

In [13]:
#text_with_pytesseract = extract_text_from_img("/content/Screenshot 2025-01-17 at 17.45.18.png")
#print(text_with_pytesseract)

# Prompting LLMs

In [14]:
def generate_pdf_prompt(text):
  # extract text from pdf
  #extracted_text = extract_text(pdf_file)

  if not text:
        return "No text found in the PDF. Please upload a different file."

  prompt = f"""
  Here is the extracted text from the document, please go throgh it and help me improve it:
  {text}
  """
  return prompt

In [15]:
def extract_assistant_message(text):
    parts = text.rsplit("Assistant:", 1)
    if len(parts) > 1:
        return parts[1].strip()
    return text.strip()

## Prompting for User's PDF

In [16]:
def analyze_user_resume(text):
    # Generate prompt
    user_prompt = generate_pdf_prompt(text)

    system_prompt = """
    You are a helpful assistant that knows all about recruiting, you do the following:
    - receive resume/cv document
    - analyzes the contents in the resume and returns helpful feedback like:
    mispelt words, better use of grammar, summarizing texts into bullet points
    where needed, but not limited to these alone.
    - provide useful feedback in form of bullet points, reference the original text from the document if needed.
    - in your final feedback return the resume back to the user as it was in the markdown format with the changes made, indicate the places where the changes were made with a bracket at the end of the sentence like this (AI).
    """
    gemma_prompt = system_prompt + "\n\n" + user_prompt
    messages = [
        #{"role": "system", "content": system_prompt},
        {"role": "user", "content": gemma_prompt}
    ]

    try:
        # Apply chat template
        inputs = tokenizer.apply_chat_template(
            messages,
            return_tensors='pt',
            add_generation_prompt=True,
            return_dict=True,
            return_attention_mask=True
        )

        # Move to CUDA if available
        if torch.cuda.is_available():
            inputs = inputs.to('cuda')

    except Exception as e:
        # Fallback approach
        combined_text = f"{system_prompt}\n\nUser: {user_prompt}\n\nAssistant:"
        inputs = tokenizer(combined_text, return_tensors='pt', padding=True)

        # Move to CUDA if available
        if torch.cuda.is_available():
            inputs = inputs.to('cuda')

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        GEMMA2,
        device_map="auto",
        quantization_config=quant_config
    )

    # Generate output
    with torch.no_grad():  # Disable gradient tracking to save memory
        output = model.generate(
            input_ids=inputs.get('input_ids', inputs), # If 'input_ids' key exists, use it, otherwise use the whole inputs
            max_new_tokens=1500,
            attention_mask=inputs['attention_mask'],
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode output
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    result = extract_assistant_message(decoded_output)

    # Explicitly free memory
    del model, inputs, output
    torch.cuda.empty_cache()

    return result

In [17]:
#print(analyze_user_resume("/content/ODESANYA KONYINSOLA'S RESUME.pdf"))

## Getting Online Resumes

In [18]:
def get_resumes_from_web(company_name, job_type):
    search_input = f"{job_type} {company_name} resume template"
    base_output_dir = "/content/online_resumes"

    # Create base directory
    os.makedirs(base_output_dir, exist_ok=True)

    online_resumes = []

    try:
        # Download images
        downloaded_files = downloader.download(
            search_input,
            limit=3,
            output_dir=base_output_dir,
            adult_filter_off=True,
            force_replace=False,
            timeout=60
        )

        # The downloader creates a subfolder with the search query name
        # We need to find this subfolder
        expected_subfolder = os.path.join(base_output_dir, search_input)

        # Check if the expected subfolder exists
        if os.path.exists(expected_subfolder) and os.path.isdir(expected_subfolder):
            print(f"Found subfolder: {expected_subfolder}")

            # Get all image files in the subfolder
            image_files = [os.path.join(expected_subfolder, f) for f in os.listdir(expected_subfolder)
                          if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

            print(f"Found {len(image_files)} image files")

            # Process each image file
            for i, file_path in enumerate(image_files):
                try:
                    print(f"Processing image {i+1}: {file_path}")

                    # Open the image file
                    img = Image.open(file_path)

                    # Extract text from the image
                    extracted_text = extract_text_from_img(img)

                    if extracted_text:
                        print(f"Successfully extracted text from image {i+1}")
                        online_resumes.append(extracted_text)
                    else:
                        print(f"No text extracted from image {i+1}")

                except Exception as e:
                    print(f"Error processing image {i+1}: {e}")
        else:
            print(f"Expected subfolder not found: {expected_subfolder}")

            # Look for alternative subfolders
            subfolders = [d for d in os.listdir(base_output_dir)
                         if os.path.isdir(os.path.join(base_output_dir, d))]

            if subfolders:
                print(f"Found alternative subfolders: {subfolders}")

                # Process the first alternative subfolder found
                alt_folder = os.path.join(base_output_dir, subfolders[0])
                image_files = [os.path.join(alt_folder, f) for f in os.listdir(alt_folder)
                              if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

                # Process images in alternative folder
                for i, file_path in enumerate(image_files):
                    try:
                        img = Image.open(file_path)
                        extracted_text = extract_text_from_img(img)

                        if extracted_text:
                            online_resumes.append(extracted_text)
                    except Exception as e:
                        print(f"Error processing image in alternative folder: {e}")

    except Exception as e:
        print(f"Error during download process: {e}")

    finally:
        # Clean up - remove the base directory and all its contents
        if os.path.exists(base_output_dir):
            try:
                shutil.rmtree(base_output_dir)
                print(f"Cleaned up temporary directory: {base_output_dir}")
            except Exception as e:
                print(f"Error cleaning up directory: {e}")

    print(f"Total resumes extracted: {len(online_resumes)}")
    return online_resumes

In [19]:
#online_resumes = get_resumes_from_web("Google", "Software Developer")
#print(online_resumes)

In [20]:
def generate_online_resume_prompt(company_name, job_type, resume_text):
    messages = [
        #{"role": "system", "content": f"You are a helpful assistant that knows all about recruiting for {company_name} for the job role of {job_type}, you have received a resume from the internet that are similar to the job the user wants to apply for."},
        {"role": "user", "content": f"""
      You are a helpful assistant that knows all about recruiting for {company_name} for the job role of {job_type}, you have received a resume from the internet that are similar to the job the user wants to apply for.
      You have received a resume from the internet that is similar to the job the user wants to apply for.
      Your task is to:
      - Gather important points/aspects used in the resume that will be useful to give insights to a user.
      - Look for important keywords used relating to the '{job_type}' role.
      - Return the response in a format that preserves the online resume as it will be used for comparison and similarity checking.

      Here is the extracted text from the document:
      {resume_text}
      """}
          ]

    return messages

### Prompting for Online Resumes

In [21]:
def analyze_online_resumes(company_name, job_type):
    online_resumes = get_resumes_from_web(company_name, job_type)

    if not online_resumes:
        print("No valid resumes found to analyze")
        return []

    insight = []

    try:
        # Load model once
        model = AutoModelForCausalLM.from_pretrained(
            GEMMA2,
            device_map="auto" if torch.cuda.is_available() else None,
            quantization_config=quant_config
        )

        for i, resume in enumerate(online_resumes):
            if not resume:
                print(f"Skipping empty resume #{i+1}")
                continue

            try:
                print(f"Processing resume #{i+1}")
                messages = generate_online_resume_prompt(company_name, job_type, resume)

                # Process inputs with proper error handling
                try:
                    inputs = tokenizer.apply_chat_template(
                        messages,
                        return_tensors='pt',
                        add_generation_prompt=True,
                        return_dict=True,
                        return_attention_mask=True
                    )

                    if torch.cuda.is_available():
                        inputs = {k: v.to('cuda') for k, v in inputs.items()}

                except Exception as e:
                    print(f"Chat template error for resume #{i+1}: {e}")
                    combined_text = f"System: {messages[0]['content']}\n\nUser: {messages[1]['content']}\n\nAssistant:"
                    inputs = tokenizer(combined_text, return_tensors='pt', padding=True)

                    if torch.cuda.is_available():
                        inputs = {k: v.to('cuda') for k, v in inputs.items()}

                # Generate with torch.no_grad() to save memory
                try:
                    with torch.no_grad():
                        output = model.generate(
                            input_ids=inputs.get('input_ids', inputs), # If 'input_ids' key exists, use it, otherwise use the whole inputs
                            max_new_tokens=1500,
                            attention_mask=inputs['attention_mask'],
                            pad_token_id=tokenizer.eos_token_id
                        )

                    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
                    processed_output = extract_assistant_message(decoded_output)

                    if processed_output:
                        insight.append(processed_output)
                        print(f"Successfully processed resume #{i+1}")
                    else:
                        print(f"No valid output generated for resume #{i+1}")

                    # Free memory after each resume processing
                    del output, inputs, decoded_output
                    torch.cuda.empty_cache()

                except Exception as e:
                    print(f"Generation error for resume #{i+1}: {e}")
                    # Free any partial resources
                    if 'inputs' in locals():
                        del inputs
                    if 'output' in locals():
                        del output
                    torch.cuda.empty_cache()
                    continue

            except Exception as e:
                print(f"Processing error for resume #{i+1}: {e}")
                continue

        # Free model memory after all processing is complete
        del model
        torch.cuda.empty_cache()

    except Exception as e:
        print(f"Model loading error: {e}")
        return []

    print(f"Successfully analyzed {len(insight)} out of {len(online_resumes)} resumes")
    return insight

In [22]:
#print(analyze_online_resumes("Google", "Software Developer"))

# Final Prompting -- Combining Both User and Online Resume Insight

In [23]:
def generate_resume_insight_prompt(user_resume_insight, formatted_online_insights):
    prompt = f"""
      Here is the insight from the user's resume:
      {user_resume_insight}

      Here are the insights from the online resumes:
      {formatted_online_insights}

      Tailor the resume of the user based on information gotten from online resumes.
      """

    return prompt

In [24]:
def get_resume_insight(pdf_file, company_name, job_type):
    user_resume_insight = analyze_user_resume(pdf_file)
    online_resume_insight = analyze_online_resumes(company_name, job_type)

    formatted_online_insights = "\n\n".join(online_resume_insight)

    system_prompt = f"""
    You are a helpful assistant that knows all about recruiting for {company_name} for the job role of {job_type}.
    You have received insights from the user's resume and multiple insights from online resumes.

    You are to:
    - Compare and gather similarities from all the online resumes.
    - Use that to improve the user's resume for the '{job_type}' role.
    - Do NOT add any activity/project/experience that is not already present in the user's resume.
    - In your final feedback return the resume back to the user as it was in the markdown format with the changes made, indicate the places where the changes were made with an asterick or so.
    Provide the result in **markdown format**.
    """

    # Get user prompt
    user_prompt = generate_resume_insight_prompt(user_resume_insight, formatted_online_insights)

    # Free memory from previous steps
    if 'user_resume_insight' in locals():
        del user_resume_insight
    if 'online_resume_insight' in locals():
        del online_resume_insight
    torch.cuda.empty_cache()

    gemma_prompt = system_prompt + "\n\n" + user_prompt
    messages = [
        {"role": "user", "content": gemma_prompt}
    ]

    try:
        # Load model
        model = AutoModelForCausalLM.from_pretrained(
            GEMMA2,
            device_map="auto",
            quantization_config=quant_config
        )

        # Apply chat template
        inputs = tokenizer.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True,
            return_dict=True,
            return_attention_mask=True
        )

        # Move to CUDA if available
        if torch.cuda.is_available():
            inputs = inputs.to("cuda")

        # Setup streamer
        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        # Generate in a separate thread to enable streaming
        generation_kwargs = {
            "input_ids": inputs.get('input_ids', inputs),
            "attention_mask": inputs.get('attention_mask', None),
            "max_new_tokens": 1500,
            "streamer": streamer,
            "pad_token_id": tokenizer.eos_token_id
        }

        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

        # Yield output for streaming in Gradio
        generated_text = ""
        for text_chunk in streamer:
            generated_text += text_chunk
            # This is the key part for Gradio streaming
            yield generated_text

        thread.join()

        # Free memory
        del model, inputs, streamer
        torch.cuda.empty_cache()

    except Exception as e:
        error_msg = f"An error occurred while processing the resume: {str(e)}"
        print(f"Error in get_resume_insight: {e}")
        # Clean up in case of error
        if 'model' in locals():
            del model
        if 'inputs' in locals():
            del inputs
        if 'streamer' in locals():
            del streamer
        torch.cuda.empty_cache()

        yield error_msg

In [25]:
def get_resume_insight_interface(file_path, company_name, job_type):
    resume_text = extract_text(file_path)  # Extract raw text from file
    return get_resume_insight(resume_text, company_name, job_type)  # Call your core logic

# Gradio Interface

In [26]:
with gr.Blocks() as ui:
  gr.Markdown("# Upload Your Resume/CV and get valuable insight on how to improve it 😉")
  with gr.Row():
    with gr.Column(variant='panel', scale=5):
      file = gr.File(label="Please upload a PDF or Word Document of your resume", file_types=[".pdf", ".docx"])
      company_name = gr.Textbox(label = "Input name of company intrested in applying to", placeholder = "e.g. Google, Amazon, KPMG, ARCO", value = "Google")
      job_role = gr.Textbox(label = "Input intrested job role", placeholder = "e.g. Software Developer, Accountant, Business Analyst", value = "Software Developer")
      with gr.Row():
        generate_btn = gr.Button("Generate")
        clear_btn = gr.ClearButton(value="Clear")
      show_pdf = PDF(label='Document preview', interactive=False, visible=True, height=800)

    with gr.Column(variant='panel', scale=5):
      output = gr.Markdown(label="Generated Output", height=1100, show_copy_button=True, line_breaks=True)

  file.change(fn=display_pdf, inputs=file, outputs=show_pdf)
  generate_btn.click(fn=get_resume_insight_interface, inputs=[file, company_name, job_role], outputs=output)
  clear_btn.add([file, company_name, job_role, output, show_pdf])

ui.launch(inbrowser=True, debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://359550666c2fd18a87.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[%] Downloading Images to /content/online_resumes/Software Developer Google resume template


[!!]Indexing page: 1

[%] Indexed 3 Images on Page 1.


[%] Downloading Image #1 from https://www.hloom.com/images/Boast-gdoc.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://cdn.resumecatstatic.com/resume-examples/google-software-developer-resume-example-OPEfo-1600w.png
[%] File Downloaded !

[%] Downloading Image #3 from https://as2.ftcdn.net/v2/jpg/05/75/51/51/1000_F_575515196_ljyVLoydCv5ovTDS1n22PuRqar4xERj6.jpg
[%] File Downloaded !



[%] Done. Downloaded 3 images.
Found subfolder: /content/online_resumes/Software Developer Google resume template
Found 3 image files
Processing image 1: /content/online_resumes/Software Developer Google resume template/Image_2.png
Successfully extracted text from image 1
Processing image 2: /content/online_resumes/Software Developer Google resume template/Image_1.jpg
Successfully extracted text from image 2
Processing image 3: /content/onli

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processing resume #1
Successfully processed resume #1
Processing resume #2
Successfully processed resume #2
Processing resume #3
Successfully processed resume #3
Successfully analyzed 3 out of 3 resumes


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You're right! I apologize for the previous response. I was focusing on the general structure of resumes and not on the specific details needed for a Software Developer role. 

Here's a breakdown of the user's resume and how to improve it, along with some additional tips for a successful application:

**Analysis of the User's Resume:**

* **Missing Information:**  The resume lacks crucial details like:
    * **Full Name:**  This is essential for professional communication.
    * **Contact Information:**  Phone number, email address, and LinkedIn profile are crucial.
    * **Education:**  Degree, university, major, and relevant coursework.
    * **Technical Skills:**  Programming languages, frameworks, tools, and technologies.
    * **Projects:**  Personal or academic projects that demonstrate skills.
    * **Experience:**  Specific job titles, responsibilities, and accomplishments.
* **Generic Language:**  The resume uses vague terms and lacks specific examples.
* **No Keywords:**  The 

# FALLBACK CODE

In [27]:
def get_resume_insight_old(pdf_file, company_name, job_type):
    user_resume_insight = analyze_user_resume(pdf_file)
    online_resume_insight = analyze_online_resumes(company_name, job_type)

    formatted_online_insights = "\n\n".join(online_resume_insight)

    system_prompt = f"""
    You are a helpful assistant that knows all about recruiting for {company_name} for the job role of {job_type}.
    You have received insights from the user's resume and multiple insights from online resumes.

    You are to:
    - Compare and gather similarities from all the online resumes.
    - Use that to improve the user's resume for the '{job_type}' role.
    - Do NOT add any activity/project/experience that is not already present in the user's resume.
    - In your final feedback return the resume back to the user as it was in the markdown format with the changes made, indicate the places where the changes were made with an asterick or so.

    Provide the result in **markdown format**.
    """

    # Get user prompt
    user_prompt = generate_resume_insight_prompt(user_resume_insight, formatted_online_insights)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Tokenize with attention mask
    inputs = tokenizer.apply_chat_template(
        messages,
        return_tensors='pt',
        return_attention_mask=True
    ).to('cuda')

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        mistral,
        device_map="auto",
        quantization_config=quant_config
    )

    # Prepare streamer
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],  # pass attention_mask
        "max_new_tokens": 1500,
        "streamer": streamer,
        "pad_token_id": tokenizer.eos_token_id  # to avoids warnings
    }

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    full_output = ""
    for text_chunk in streamer:
        print(text_chunk, end="", flush=True)
        full_output += text_chunk

    thread.join()
    return full_output

In [28]:
def get_resumes_from_web_old(company_name, job_type):
  search_input = f"{job_type} {company_name} resume template"
  results = search(search_input, max_results=3)
  print(f"--RESULTS-- {results}")
  if not results:
    print("No images found.")
    return []

  online_resumes = []
  for i, result in enumerate(results):
    print(f"FOR LOOP RESULT -- {result}")
    print(f"\n--- Image {i+1} ---")
    image_url = result['results'][i]['image']
    print(f"Image URL: {image_url}")

    try:
      # Download and open image
      response = requests.get(image_url)
      img = Image.open(BytesIO(response.content)) # creates an in-memory binary stream to prevent writing to disk

      # Converting the downloaded images into markdown and inserting them into a list
      online_resumes.append(extract_text_from_img(img))
    except Exception as e:
      print(f"Error processing image {i+1}: {e}")
      # exception to handle bad processing of an image
      online_resumes.append(None)
  return online_resumes

In [29]:
def analyze_online_resumes_old(company_name, job_type):
    online_resumes = get_resumes_from_web(company_name, job_type)

    # model running outside loop so as to conserve space and GPU processing
    model = AutoModelForCausalLM.from_pretrained(mistral, device_map="auto", quantization_config=quant_config)

    insight = []
    for resume in online_resumes:
        if not resume:
            continue  # Skip if empty, but continue with others

        messages = generate_online_resume_prompt(company_name, job_type, resume)

        input = tokenizer.apply_chat_template(messages, return_tensors='pt', return_attention_mask=True).to('cuda')

        output = model.generate(
            input_ids=input["input_ids"],  # <-- pass input_ids explicitly
            attention_mask=input["attention_mask"],  # <-- pass attention_mask
            max_new_tokens=1500,
            pad_token_id=tokenizer.eos_token_id  # <-- to avoid warnings
        )
        insight.append(extract_after_last_inst(tokenizer.decode(output[0])))

    return insight

In [30]:
def analyze_user_resume_old(text):
    # Generate string prompt
    user_prompt = generate_pdf_prompt(text)

    system_prompt = """
    You are a helpful assistant that knows all about recruiting, you do the following:
    - receive resume/cv document
    - analyzes the contents in the resume and returns helpful feedback like:
    mispelt words, better use of grammar, summarizing texts into bullet points
    where needed, but not limited to these alone.
    - provide useful feedback in form of bullet points, reference the original text from the document if needed.
    - in your final feedback return the resume back to the user as it was in the markdown format with the changes made, indicate the places where the changes were made with a bracket at the end of the sentence like this (AI).
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Correctly handle the tokenizer output
    # First check if tokenizer supports chat templates
    try:
        # Get the input tensors - most modern tokenizers return a dictionary
        inputs = tokenizer.apply_chat_template(
            messages,
            return_tensors='pt',
            add_generation_prompt=True,
            return_dict=True  # Explicitly request a dictionary
        )

        # Move to CUDA if available
        if torch.cuda.is_available():
            inputs = {k: v.to('cuda') for k, v in inputs.items()}

    except Exception as e:
        # Fallback approach if the chat template method fails
        combined_text = f"{system_prompt}\n\nUser: {user_prompt}\n\nAssistant:"
        inputs = tokenizer(combined_text, return_tensors='pt', padding=True)

        # Move to CUDA if available
        if torch.cuda.is_available():
            inputs = {k: v.to('cuda') for k, v in inputs.items()}

    # Load model (only once)
    model = AutoModelForCausalLM.from_pretrained(GEMMA2, device_map="auto", quantization_config=quant_config)
    #model = AutoModelForCausalLM.from_pretrained(GEMMA2, device_map="auto")

    # Generate output
    output = model.generate(
        **inputs,  # Pass all inputs at once
        max_new_tokens=1500,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and return only the generated portion
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # This is model/tokenizer dependent
    return extract_assistant_message(decoded_output)